## Training of Mask R-CNN

The training of the algorithm can be done in the terminal. 

```bash
# cd to the MLtools GIT repository
cd projects/bouldering/
python train_net.py --help # if you want to have information about the different inputs

usage: train_net.py [-h] [--config-file FILE] [--resume] [--eval-only] [--num-gpus NUM_GPUS] [--num-machines NUM_MACHINES] [--machine-rank MACHINE_RANK] [--aug-path FILE]
                    [--min-area-npixels MIN_AREA_NPIXELS] [--optimizer-name OPTIMIZER_NAME] [--scheduler-mode SCHEDULER_MODE] [--dist-url DIST_URL]
                    ...

positional arguments:
  opts                  Modify config options at the end of the command. For Yacs configs, use space-separated "PATH.KEY VALUE" pairs. For python-based LazyConfig, use "path.key=value".

optional arguments:
  -h, --help            show this help message and exit
  --config-file FILE    path to config file
  --resume              Whether to attempt to resume from the checkpoint directory. See documentation of `DefaultTrainer.resume_or_load()` for what it means.
  --eval-only           perform evaluation only
  --num-gpus NUM_GPUS   number of gpus *per machine*
  --num-machines NUM_MACHINES
                        total number of machines
  --machine-rank MACHINE_RANK
                        the rank of this machine (unique per machine)
  --aug-path FILE       path to augmentation file
  --min-area-npixels MIN_AREA_NPIXELS
                        threshold for filtering masks smaller than X pixels
  --optimizer-name OPTIMIZER_NAME
                        optimizer name (SGD, ADAM, ADAMW)
  --scheduler-mode SCHEDULER_MODE
                        mode of cycling in CyclingLR
  --dist-url DIST_URL   initialization URL for pytorch distributed backend. See https://pytorch.org/docs/stable/distributed.html for details.

```

So far, only `SGD` works in `--optimizer-name`

## How to create an augmentation file

In [3]:
import albumentations as A
import pandas as pd
from pathlib import Path

In [4]:
home_p = Path.home()
test_aug_p = home_p / "tmp" / "BOULDERING" / "best_model" / "test_augmentation_setup.json"

In order to compensate for the relatively small amount of image patches used in our study, we had to use a large number of augmentations. We organize it in number of blocks:
1. Rotations of the image
2. Contrast, Brightness/intensity, Noise/Blur
3. Re-scaling to larger size and cropping down to 512 pixels in 50% if time.
4. Elastic transform, Grid distortion, coarse dropout.

If you are unfamiliar with augmentations, please check the documentation of Albumentation (https://albumentations.ai/).

In [5]:
default_transform = A.LongestMaxSize(max_size=512, interpolation=2, p=1.0)

block1 = A.OneOf([A.NoOp(p=1.0), 
                  A.Affine(p=1.0, rotate = 90.0), 
                  A.Affine(p=1.0, rotate = 180.0), 
                  A.Affine(p=1.0, rotate = 270.0),
                  A.HorizontalFlip(p=1.0), 
                  A.VerticalFlip(p=1.0), 
                  A.Transpose(p=1.0), 
                  A.Compose([A.Affine(p=1.0, rotate = 180.0), A.Transpose(p=1.0)], p=1.0)], p=1.0)

block1_and_default = A.Compose([default_transform, block1])

block2 = A.OneOf([A.CLAHE(p=1.0, clip_limit=(1.0, 4.0), tile_grid_size=(8,8)), 
                  A.RandomGamma(p=1.0, gamma_limit=(80, 150)), 
                  A.GaussNoise(p=1.0, var_limit=(10.0, 50.0), mean=0, per_channel=False), 
                  A.Sharpen(p=1.0),
                  A.Blur(p=1.0, blur_limit=(3,7)),
                  A.NoOp(p=1.0)], p=1.0)

block12_and_default = A.Compose([default_transform, block1, block2])

block3_p1 = A.Compose([A.LongestMaxSize(max_size=[640, 768, 896, 1024, 1152, 1280], interpolation=2, p=1.0), 
                       A.CropNonEmptyMaskIfExists(height=512, width=512, p=0.5)], p=1.0) # 256?
block3_p2 = A.NoOp(p=0.5)

block3 = A.OneOf([block3_p1, block3_p2], p=1.0)
                  
block123_and_default = A.Compose([default_transform, block1, block2, block3])

block4 = A.OneOf([A.ElasticTransform(p=1.0, alpha=1, sigma=50, alpha_affine=20, interpolation=2, border_mode=0), 
                  A.GridDistortion(p=1.0, num_steps=8, distort_limit=0.3, interpolation=2, border_mode=0), 
                  A.CoarseDropout(p=1.0, min_holes=2, max_holes=32, max_height=12, min_height=12, min_width=12, max_width=12), 
                  A.NoOp(p=1.0)], p=1.0)

block1234_and_default = A.Compose([default_transform, block1, block2, block3, block4])
pd.DataFrame(block1234_and_default.to_dict()).to_json(test_aug_p)

In [6]:
block1234_and_default

Compose([
  LongestMaxSize(always_apply=False, p=1.0, max_size=512, interpolation=2),
  OneOf([
    NoOp(always_apply=False, p=1.0),
    Affine(always_apply=False, p=1.0, interpolation=1, mask_interpolation=0, cval=0, mode=0, scale={'x': (1.0, 1.0), 'y': (1.0, 1.0)}, translate_percent=None, translate_px={'x': (0, 0), 'y': (0, 0)}, rotate=(90.0, 90.0), fit_output=False, shear={'x': (0.0, 0.0), 'y': (0.0, 0.0)}, cval_mask=0, keep_ratio=False, rotate_method='largest_box'),
    Affine(always_apply=False, p=1.0, interpolation=1, mask_interpolation=0, cval=0, mode=0, scale={'x': (1.0, 1.0), 'y': (1.0, 1.0)}, translate_percent=None, translate_px={'x': (0, 0), 'y': (0, 0)}, rotate=(180.0, 180.0), fit_output=False, shear={'x': (0.0, 0.0), 'y': (0.0, 0.0)}, cval_mask=0, keep_ratio=False, rotate_method='largest_box'),
    Affine(always_apply=False, p=1.0, interpolation=1, mask_interpolation=0, cval=0, mode=0, scale={'x': (1.0, 1.0), 'y': (1.0, 1.0)}, translate_percent=None, translate_px={'x': (0,